# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [210]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from credentials import *

In [211]:
conn = psycopg2.connect("host={} dbname={} user={} password={}".format(host,dbname,user,password))
cur = conn.cursor()

In [212]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [213]:
song_files = get_files("data/song_data")
print(song_files)

['C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\

In [214]:
filepath =  song_files[1]
print(filepath)

C:\Users\sthoe09258\OneDrive - Elekta\Desktop\udacity\DataEngineering\project1\project-template\data\song_data\A\A\A\TRAAABD128F429CF47.json


In [215]:
#df = pd.read_json(filepath, typ='series')
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [216]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data

array(['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969,
       148.03546], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [217]:
song_table_insert = ("""INSERT INTO songs(song_id, title, artist_id, year, duration)
VALUES(%s, %s, %s, %s, %s)
""")
song_table_insert
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [218]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]
artist_data

array(['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968,
       -90.04892], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
artist_table_insert = """INSERT INTO ARTISTS (artist_id ,name ,location , latitude ,longitude  ) VALUES(%s, %s,%s, %s,%s) ON CONFLICT (artist_id) DO NOTHING"""
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")
log_files

In [ ]:
filepath = log_files[0]
filepath

In [ ]:
df = pd.read_json(filepath, lines=True )
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df[df['page'] == 'NextSong']
df.head()

In [ ]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

In [ ]:
time_data = pd.concat([t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday], axis = 1)
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
time_data.head()
#column_labels = ()

In [ ]:
time_df = pd.DataFrame(data=time_data.values, columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
time_table_insert = ("""
INSERT INTO time(start_time, hour, day, week, month, year, weekday)
VALUES(%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (start_time) DO NOTHING
""")

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
user_table_insert = ("""
INSERT INTO users(user_id, first_name, last_name, gender, level)
VALUES(%s, %s, %s, %s, %s)
ON CONFLICT (user_id) DO UPDATE set level = EXCLUDED.level
""")
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
song_select = """select s.song_id songid, s.artist_id artistid from songs s inner join artists a on s.artist_id = a.artist_id where s.title = %s and a.name = %s and s.duration = %s"""
songplay_table_insert = """INSERT INTO songplays(start_time,user_id,level,song_id,artist_id,session_id,location,user_agent) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"""


for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.